<a href="https://colab.research.google.com/github/sahiliem/colab/blob/main/Building_a_Financial_Advisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Apr  9 14:30:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install openai pytube -q
!pip install git+https://github.com/openai/whisper.git -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 30.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.1 MB/s eta 0:00:00


In [4]:
import openai
import whisper
import pandas as pd
from pytube import YouTube
from getpass import getpass

COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDINGS_MODEL = "text-embedding-ada-002"
openai.api_key = getpass("Enter your Pass")

Enter your Pass··········


In [5]:
model = whisper.load_model('base')

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 64.1MiB/s]


In [6]:
youtube_video_url = "https://www.youtube.com/watch?v=OYxhrPdZdog"
youtube_video = YouTube(youtube_video_url)

In [7]:
dir(youtube_video)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_age_restricted',
 '_author',
 '_embed_html',
 '_fmt_streams',
 '_initial_data',
 '_js',
 '_js_url',
 '_metadata',
 '_player_config_args',
 '_publish_date',
 '_title',
 '_vid_info',
 '_watch_html',
 'age_restricted',
 'allow_oauth_cache',
 'author',
 'bypass_age_gate',
 'caption_tracks',
 'captions',
 'channel_id',
 'channel_url',
 'check_availability',
 'description',
 'embed_html',
 'embed_url',
 'fmt_streams',
 'from_id',
 'initial_data',
 'js',
 'js_url',
 'keywords',
 'length',
 'metadata',
 'publish_date',
 'rating',
 'register_on_complete_callback',
 'register_on_progress_callback',
 'stream_monostate',
 'streamin

In [8]:
stream = youtube_video.streams.filter(only_audio=True).first()
stream.download(filename='financial_advisor.mp4')

'/content/financial_advisor.mp4'

In [9]:
youtube_video.description

"Welcome to Portfolio Rescue, the show where we answer your questions about investing and personal finance. On this episode, Ben and Duncan are joined by RWM advisor, Matt Lohrius. Submit your questions to askthecompoundshow@gmail.com!\n\n► 00:00 - Intro\n► 01:58 - How does the fed raising interest rates fights off inflation?\n► 03:30 - What particular sectors/names do you find most compelling for a young investor with a 35-40 year time horizon?\n► 05:16 - What will bring more value in the long run for a child?\n► 10:18 - Would you recommend focusing on paying off my student loans first? Or should I begin building my portfolio? \n► 11:57 - Paying off a mortgage.\n► 13:25 - What happens when the Fed raises rates?\n► 14:39 - I’m 44 and have a net worth of around $425K. Would have more if TWTR didn’t crash, I didn’t sell any - I own 5,550 shares at @ $29.83 cost basis. What do you think about where I am? \n► 17:19 - What are some good questions to ask when selecting a financial advisor, e

In [10]:
output = model.transcribe("financial_advisor.mp4")

In [11]:
output['text']

" Welcome back to Portfolio Rescue. I'll show you where we take questions straight from you. We've got a lot of questions like I said last week. We're going to try to do a lightning round today. Before we get to that, I remember asked the compound show at gmail.com Duncan. Last week's show I mentioned the 50-30-20 rule. I said, I don't know where it comes from. A lot of these things in finance, they just sound good and they stick like the 60-40 Portfolio. No one actually knows who started that or where it came from. We all just kind of subscribed to it. But the 50-30-20 rule. Last Thursday after we did the show, I was reading a book at night, kind of read before I go to bed. I was reading this book called Die Was Zero by a guy named Bill Perkins. The whole idea is if you have a lot of money, your hope should be to spend that money, or give it away to charity or spend it all on experiences before you die. Interesting concept. A lot. You were talking about animal spirits. Yeah, one of th

In [14]:
import csv

# create a new pandas dataframe
df = pd.DataFrame(columns=['episode', 'url', 'start_timestamp', 'start', 'end', 'question', 'context'])

# index to keep track of current row
i = 0

with open('/content/Financial Advisor.csv') as csv_file:
    reader = csv.reader(csv_file)

    # skip the header row in the csv file
    next(reader)
    
    for row in reader:
        # assign each column in the row to a variable and split questions on carriage return
        episode, url, questions = row
        question_list = questions.split("\n")

        # for each question in the list, extract the timestamp and convert it to seconds for youtube
        for question in question_list:
          pieces = question.split('-')
          timestamp = pieces[0]
          minutes, seconds = timestamp.split(':')
          seconds = int(seconds) + (int(minutes.lstrip()) * 60)

          # add a new row to the dataframe
          df.loc[i] = [episode, url, timestamp, seconds, seconds, " ".join(pieces[1:]), ""]
          
          try:
            df.loc[i-1]['end'] = df.loc[i]['start']
          except:
            print(f"skipping row {i} because there is no previous row")

          i+=1

          df['end'][df['end'] < df['start']] = 0
          df['end'][df['start'] == df['end']] = 0

df.to_csv("/content/questions.csv")


skipping row 0 because there is no previous row


<ipython-input-14-e972049255c8>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['end'][df['end'] < df['start']] = 0
<ipython-input-14-e972049255c8>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['end'][df['start'] == df['end']] = 0
<ipython-input-14-e972049255c8>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i-1]['end'] = df.loc[i]['start']
<ipython-input-14-e972049255c8>:37: SettingWithCopyWarning: 
A value is tr

In [15]:
import pandas as pd

df = pd.read_csv('/content/questions.csv')
df

,Unnamed: 0,episode,url,start_timestamp,start,end,question,context
0,0,Episode 1,https://www.youtube.com/watch?v=0z49u_b6PmE,03:07,187,0,What's the best way to reallocate a portfolio...,NaN
1,1,Episode 1,https://www.youtube.com/watch?v=0z49u_b6PmE,06:55,415,0,What do I do once I cross the Roth IRA income...,NaN
2,2,Episode 1,https://www.youtube.com/watch?v=0z49u_b6PmE,11:09,669,0,Once I have maxed out my retiremenet accounts...,NaN
3,3,Episode 2,https://www.youtube.com/watch?v=IDGnZ2AErBg,01:04,64,0,Saving vs. Investing,NaN
4,4,Episode 2,https://www.youtube.com/watch?v=IDGnZ2AErBg,06:06,366,0,If Baby Boomers are shifting from net investo...,NaN
...,...,...,...,...,...,...,...,...
323,323,Episode 67,https://www.youtube.com/watch?v=vh7XLSLCpKQ,03:00,180,0,Did VCs cause the bank run on Twitter?,NaN
324,324,Episode 67,https://www.youtube.com/watch?v=vh7XLSLCpKQ,08:44,524,0,What happens if a brokerage fails?,NaN
325,325,Episode 67,https://www.youtube.com/watch?v=vh7XLSLCpKQ,16:00,960,0,FHLB.,NaN
326,326,Episode 67,https://www.youtube.com/watch?v=vh7XLSLCpKQ,20:32,1232,0,Are Money Market Funds safe?,NaN


In [16]:
# Let's just get the questions for a single episode and make this work before we download and transcribe all episodes in bulk
episode = df[df['episode'] == 'Episode 22'].copy()
episode

,Unnamed: 0,episode,url,start_timestamp,start,end,question,context
83,83,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,01:58,118,0,How does the fed raising interest rates fight...,NaN
84,84,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,03:30,210,0,What particular sectors/names do you find mos...,NaN
85,85,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,05:16,316,0,What will bring more value in the long run fo...,NaN
86,86,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,10:18,618,0,Would you recommend focusing on paying off my...,NaN
87,87,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,11:57,717,0,Paying off a mortgage.,NaN
88,88,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,13:25,805,0,What happens when the Fed raises rates?,NaN
89,89,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,14:39,879,0,I’m 44 and have a net worth of around $425K. ...,NaN
90,90,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,17:19,1039,0,What are some good questions to ask when sele...,NaN
91,91,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,21:15,1275,0,"At interest rates so low, should I instead t...",NaN


In [17]:
def is_part_of_question(segment, start, end):
    if segment['start'] > start:
      if segment['end'] < end or end == 0:
          return True 

    return False

def get_question_context(row):
     global output

     question_segments = list(filter(lambda segment: is_part_of_question(segment, row['start'], row['end']), output['segments']))
     # include question from timestamp in the context
     context = row['question']
     for segment in question_segments:
         context += segment['text']

     return context

episode['context'] = episode.apply(get_question_context, axis=1)

episode

,Unnamed: 0,episode,url,start_timestamp,start,end,question,context
83,83,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,01:58,118,0,How does the fed raising interest rates fight...,How does the fed raising interest rates fight...
84,84,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,03:30,210,0,What particular sectors/names do you find mos...,What particular sectors/names do you find mos...
85,85,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,05:16,316,0,What will bring more value in the long run fo...,What will bring more value in the long run fo...
86,86,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,10:18,618,0,Would you recommend focusing on paying off my...,Would you recommend focusing on paying off my...
87,87,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,11:57,717,0,Paying off a mortgage.,"Paying off a mortgage. For sure. Okay. So, up..."
88,88,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,13:25,805,0,What happens when the Fed raises rates?,What happens when the Fed raises rates? Okay....
89,89,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,14:39,879,0,I’m 44 and have a net worth of around $425K. ...,I’m 44 and have a net worth of around $425K. ...
90,90,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,17:19,1039,0,What are some good questions to ask when sele...,What are some good questions to ask when sele...
91,91,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,21:15,1275,0,"At interest rates so low, should I instead t...","At interest rates so low, should I instead t..."


In [18]:
from openai.embeddings_utils import get_embedding

#get_embedding(episode.iloc[0]['context'], engine='text-embedding-ada-002')

episode['embedding'] = episode['context'].apply(lambda row: get_embedding(row, engine='text-embedding-ada-002'))
episode.to_csv('/content/question_embeddings.csv')

In [19]:
from openai.embeddings_utils import cosine_similarity

question = "Should I buy a house with cash?"
question_vector = get_embedding(question, engine='text-embedding-ada-002')

episode["similarities"] = episode['embedding'].apply(lambda x: cosine_similarity(x, question_vector))
episode = episode.sort_values("similarities", ascending=False).head(4)

episode

,Unnamed: 0,episode,url,start_timestamp,start,end,question,context,embedding,similarities
91,91,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,21:15,1275,0,"At interest rates so low, should I instead t...","At interest rates so low, should I instead t...","[-0.01363264862447977, -0.011750290170311928, ...",0.851095
87,87,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,11:57,717,0,Paying off a mortgage.,"Paying off a mortgage. For sure. Okay. So, up...","[-0.013415681198239326, 0.008537850342690945, ...",0.789958
85,85,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,05:16,316,0,What will bring more value in the long run fo...,What will bring more value in the long run fo...,"[0.016351379454135895, 0.0030683542136102915, ...",0.773005
86,86,Episode 22,https://www.youtube.com/watch?v=OYxhrPdZdog,10:18,618,0,Would you recommend focusing on paying off my...,Would you recommend focusing on paying off my...,"[-0.0019429277162998915, -0.010371122509241104...",0.771841


In [20]:
episode.to_csv("/content/sorted.csv")

In [21]:
context = []
for i, row in episode.iterrows():
  context.append(row['context'])

context

["  At interest rates so low, should I instead take a loan, finance the house and invest my capital in the stock market? So last but not least, I'm in my 30s and can now afford to buy a house full cash without taking any debt. However, with interest rates so low and they wrote this one a while back, but should I take a loan anyway and finance the house and invest my capital in the stock market? I think this is interesting that rates could change the story. Going from a 3% mortgage rate to a 5% that hurdle changes. So nine months ago, I would have said you'd be nuts to pay full cash now. Maybe it kind of makes sense, but first of all, who does this person for having that amount of money? I don't really live to be able to buy and cash, but that's great. But so Matt, you're thinking through this type of decision with a client. And this is the kind of thing where there really is no right or wrong answer, right? A lot of this is personality driven and depending on what the person wants to g

In [22]:
text = "\n".join(context)
text

"  At interest rates so low, should I instead take a loan, finance the house and invest my capital in the stock market? So last but not least, I'm in my 30s and can now afford to buy a house full cash without taking any debt. However, with interest rates so low and they wrote this one a while back, but should I take a loan anyway and finance the house and invest my capital in the stock market? I think this is interesting that rates could change the story. Going from a 3% mortgage rate to a 5% that hurdle changes. So nine months ago, I would have said you'd be nuts to pay full cash now. Maybe it kind of makes sense, but first of all, who does this person for having that amount of money? I don't really live to be able to buy and cash, but that's great. But so Matt, you're thinking through this type of decision with a client. And this is the kind of thing where there really is no right or wrong answer, right? A lot of this is personality driven and depending on what the person wants to ge

In [23]:
context = text

context

"  At interest rates so low, should I instead take a loan, finance the house and invest my capital in the stock market? So last but not least, I'm in my 30s and can now afford to buy a house full cash without taking any debt. However, with interest rates so low and they wrote this one a while back, but should I take a loan anyway and finance the house and invest my capital in the stock market? I think this is interesting that rates could change the story. Going from a 3% mortgage rate to a 5% that hurdle changes. So nine months ago, I would have said you'd be nuts to pay full cash now. Maybe it kind of makes sense, but first of all, who does this person for having that amount of money? I don't really live to be able to buy and cash, but that's great. But so Matt, you're thinking through this type of decision with a client. And this is the kind of thing where there really is no right or wrong answer, right? A lot of this is personality driven and depending on what the person wants to ge

In [46]:
context = context[:20000]

context

"  At interest rates so low, should I instead take a loan, finance the house and invest my capital in the stock market? So last but not least, I'm in my 30s and can now afford to buy a house full cash without taking any debt. However, with interest rates so low and they wrote this one a while back, but should I take a loan anyway and finance the house and invest my capital in the stock market? I think this is interesting that rates could change the story. Going from a 3% mortgage rate to a 5% that hurdle changes. So nine months ago, I would have said you'd be nuts to pay full cash now. Maybe it kind of makes sense, but first of all, who does this person for having that amount of money? I don't really live to be able to buy and cash, but that's great. But so Matt, you're thinking through this type of decision with a client. And this is the kind of thing where there really is no right or wrong answer, right? A lot of this is personality driven and depending on what the person wants to ge

In [43]:
prompt = f"""Answer the following question using only the context below. Answer in the style of Ben Carlson a financial advisor and podcaster. If you don't know the answer for certain, say I don't know.

Context:
{context}

Q: {question}
A:"""

prompt

"Answer the following question using only the context below. Answer in the style of Ben Carlson a financial advisor and podcaster. If you don't know the answer for certain, say I don't know.\n\nContext:\n  At interest rates so low, should I instead take a loan, finance the house and invest my capital in the stock market? So last but not least, I'm in my 30s and can now afford to buy a house full cash without taking any debt. However, with interest rates so low and they wrote this one a while back, but should I take a loan anyway and finance the house and invest my capital in the stock market? I think this is interesting that rates could change the story. Going from a 3% mortgage rate to a 5% that hurdle changes. So nine months ago, I would have said you'd be nuts to pay full cash now. Maybe it kind of makes sense, but first of all, who does this person for having that amount of money? I don't really live to be able to buy and cash, but that's great. But so Matt, you're thinking through

In [44]:
openai.Completion.create(
    prompt=prompt,
    temperature=1,
    max_tokens=1000,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'It all depends on your personal financial situation and goals. If you have the cash to buy the house upfront and feel comfortable doing so, then you could definitely consider it. However, there are factors such as low interest rates that could be a reason to finance and invest the capital in the stock market. Ultimately, it comes down to what you feel comfortable doing with your financial situation.'